# Adding Airport arrivals

We now want to know airport arrivals becuase the users of the product tend to ride scooters from the Airport in good weather.

In [21]:
import pandas as pd
import requests
from datetime import datetime
from dotenv import load_dotenv
import os
load_dotenv()
from pytz import timezone
from datetime import datetime, timedelta
!pip install sqlalchemy
!pip install pymysql

In [10]:
# get airport information

import requests

latitude = 52.31
longitude = 13.24

base_url_w_endpoint = "https://aerodatabox.p.rapidapi.com/airports/search/location"

params = {"lat":latitude,"lon":longitude,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

headers = {
	"X-RapidAPI-Key": 'd7f3750759msh89cddae45db51b8p1f0da1jsn741fe84b90e9',
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(base_url_w_endpoint, headers=headers, params=params)
response.url


'https://aerodatabox.p.rapidapi.com/airports/search/location?lat=52.31&lon=13.24&radiusKm=50&limit=10&withFlightInfoOnly=true'

In [11]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889
1,EDDT,TXL,Berlin -Tegel,-Tegel,Berlin,DE,Europe/Berlin,52.55970,13.287699


In [17]:
def get_airports(latitudes, longitudes):

    base_url_w_endpoint = "https://aerodatabox.p.rapidapi.com/airports/search/location"
    headers = {"X-RapidAPI-Key": 'd7f3750759msh89cddae45db51b8p1f0da1jsn741fe84b90e9',
               "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}

    # DataFrame to store results
    all_airports = []

    for lat, lon in zip(latitudes, longitudes):

        base_url_w_endpoint = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        params = {"lat":lat,"lon":lon,"radiusKm":"50","limit":"10","withFlightInfoOnly":"true"}

        # Make the API request
        response = requests.get(base_url_w_endpoint, headers=headers, params=params)

        if response.status_code == 200:
            data = response.json()
            airports = pd.json_normalize(data.get('items', []))
            all_airports.append(airports)

    return pd.concat(all_airports, ignore_index=True)

In [16]:
def create_connection_string():
  schema = "Europeancities"
  host = "127.0.0.1"
  user = "root"
  password = "YOUR PASSWORD"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [19]:
def fetch_cities_data(connection_string):
  return pd.read_sql("city_info", con=connection_string)

In [18]:
city_info = fetch_cities_data(connection_string)
city_info

OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
# connection_string = create_connection_string()
city_info = fetch_cities_data(connection_string)
city_info

cities_names = fetch_cities_data(connection_string)
cities_names

city_df = cities_names.merge(city_info, left_on='city_name_id', right_on='city_name_id', how='inner')

In [ ]:
# coordinates for Berlin, Munich, Hamburg, Paris, Brussels, Freiburg
latitudes = city_df.lat.values.tolist()
latitudes

longitudes = city_df.lon.values.tolist()
longitudes

In [ ]:
airports_df = get_airports(latitudes, longitudes)
# airports_df = airports_df.drop(0).copy
airports_df

In [ ]:
airports_df = airports_df.drop(columns = 'city').copy()

MySQL  CREATE TABLE airports (
	icao VARCHAR(25) NOT NULL,
    name VARCHAR (255) NOT NULL,
    city_name_id INT NOT NULL,
    country_code_x VARCHAR (10),
    PRIMARY KEY (icao), -- primary key to id each airport
	FOREIGN KEY (city_name_id) REFERENCES city_name(city_name_id) -- for
    );

In [ ]:
airports_df = airports_df.merge(city_df, left_on='municipalityName', right_on='city', how='inner').drop(columns=[
    'municipalityName',
    'location.lat',
    'location.lon',
    'countryCode',
    'timeZone',
    'shortName',
    'city_info_id',
    'lat',
    'lon',
    'city'
    ])

In [ ]:

airports_df.to_sql('airports',
                    if_exists='append',
                    con=connection_string,
                    index=False)

In [ ]:
airports_df = pd.read_sql("airports", con=connection_string)
airports_df

In [ ]:
2) choose the information you want to keep
# flights_json = response.json()
# flights_json
# Want to keep arrival times, name of location arrival, locat time, updated times, Terminal, Status, aircraft: model, airline: name

In [20]:
schema = "Europeancities"
host = "127.0.0.1"
user = "root"
password = "PASSWORD"
port = 3306
connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

city_info = pd.read_sql("city_info", con=connection_string) # retriving information from SQL
city_info


OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 111] Connection refused)")
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/BER/2025-11-23T00:00/2025-11-23T11:59"

querystring = {
    "withLeg": "true",
    "direction": "Both",
    "withCancelled": "true",
    "withCodeshared": "false",
    "withCargo": "false",
    "withPrivate": "false",
    "withLocation": "true"
}

headers = {
    "x-rapidapi-key": "d7f3750759msh89cddae45db51b8p1f0da1jsn741fe84b90e9",
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

# print(response.status_code)
print(response.json())

In [ ]:
icao_info = airports_df.icao.to_list()
icao_info

In [ ]:
# 3) Create a loop or function to go through your airports_df

def tomorrows_flight_arrivals(icao_list):
    headers = {
    "x-rapidapi-key": "d7f3750759msh89cddae45db51b8p1f0da1jsn741fe84b90e9",
    "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
    }

    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    list_for_arrivals_df = []

    for icao in icao_list:

        times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"direction":"Arrival","withCancelled":"false"}
            headers = {"x-rapidapi-key": "d7f3750759msh89cddae45db51b8p1f0da1jsn741fe84b90e9",
                       "x-rapidapi-host": "aerodatabox.p.rapidapi.com"
                      }
            response = requests.request("GET", url, headers=headers, params=querystring)

            if response.status_code == 200:

                flights_resp = response.json()

                arrivals_df = pd.json_normalize(flights_resp["arrivals"])[["number", "airline.name", "movement.scheduledTime.local", "movement.terminal", "movement.airport.name", "movement.airport.icao"]]
                arrivals_df = arrivals_df.rename(columns={"number": "flight_number", "airline.name": "airline", "movement.scheduledTime.local": "arrival_time", "movement.terminal": "arrival_terminal", "movement.airport.name": "departure_city", "movement.airport.icao": "departure_airport_icao"})
                arrivals_df["arrival_airport_icao"] = icao
                arrivals_df["data_retrieved_on"] = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
                arrivals_df = arrivals_df[["arrival_airport_icao", "flight_number", "airline", "arrival_time", "arrival_terminal", "departure_city", "departure_airport_icao", "data_retrieved_on"]]

                # fixing arrival_time
                arrivals_df["arrival_time"] = arrivals_df["arrival_time"].str.split("+").str[0]

                list_for_arrivals_df.append(arrivals_df)

                return pd.concat(list_for_arrivals_df, ignore_index=True)
            else: continue



In [ ]:
tomorrow_flights_df_Berlin = tomorrows_flight_arrivals(icao_list)
tomorrow_flights_df_Berlin

In [ ]:
# 4) clean up table and drop information for tomorrows arrivals
airports_df
city_name_df

Airports_cities = airports_df.merge(city_name_df, left_on='city_name_id', right_on='city_name_id', how='inner').drop(columns = ['iata',
                                                                                                                                'country_code'])
Airports_cities

Incoming_flights_df = tomorrow_flights_df.merge(Airports_cities, left_on = 'arrival_airport_icao', right_on ='icao', how = 'inner').drop(columns =['city',
                                                                                                                                                  'departure_city',
                                                                                                                                                  'departure_airport_icao'])
Incoming_flights_df=Incoming_flights_df.drop(columns = 'arrival_airport_icao').copy()
# city_info = cities_from_sql.merge(city_info_from_sql, left_on='city_name_id', right_on='city_name_id', how='inner').drop(columns=[
#     'city',
#     'latitude',
#     'longitude',
#     ])
# city_info

In [ ]:
# 5) send to MYSQL
Incoming_flights_df.to_sql('flights',
                  if_exists='append',
                  con=connection_string,
                  index=False)
